In [2]:
import random
from typing import List, Tuple


P = 2**127 - 1
Q = P - 1              


G = 3
H = 5

def pedersen_commit_scalar(m: int, r: int) -> int:

    m = int(m)
    r = int(r)
    return (pow(G, m, P) * pow(H, r, P)) % P

def pedersen_commit_vector(x: List[int], s: List[int]) -> List[int]:

    assert len(x) == len(s)
    return [pedersen_commit_scalar(x[i], s[i]) for i in range(len(x))]


In [3]:
import math

def sample_l2_instance(ell: int, B2: int):


    raw = [random.randint(0, B2) for _ in range(ell)]
    norm = math.sqrt(sum(v*v for v in raw)) or 1.0
    scale = min(1.0, B2 / norm)
    x = [int(round(v * scale)) for v in raw]

    while sum(v*v for v in x) > B2*B2:
        i = random.randrange(ell)
        if x[i] > 0:
            x[i] -= 1

    s = [random.randint(0, Q-1) for _ in range(ell)]
    C = pedersen_commit_vector(x, s)
    return x, s, C

ELL = 4
B2 = 10

x_l2, s_l2, C_l2 = sample_l2_instance(ELL, B2)
print("x (l2) =", x_l2)
print("||x||_2^2 =", sum(v*v for v in x_l2))
print("C_l2 =", C_l2)


x (l2) = [4, 1, 0, 7]
||x||_2^2 = 66
C_l2 = [64561847614052468284861516350617656730, 80234625760244965677030776266106756962, 162659224838482930862925652874829768804, 111399779028978047062093727327630068257]


In [4]:
def prover_first_message_l2(x, s, B2: int):
    ell = len(x)


    mu_vec  = [random.randint(0, Q-1) for _ in range(ell)]
    rho_vec = [random.randint(0, Q-1) for _ in range(ell)]
    T = pedersen_commit_vector(mu_vec, rho_vec)

    x2    = sum(v*v for v in x)
    xplus = 2 * sum(x[i] * mu_vec[i] for i in range(ell))
    xtimes = sum(mu_vec[i] * mu_vec[i] for i in range(ell))

    s2    = random.randint(0, Q-1)
    splus = random.randint(0, Q-1)
    stimes = random.randint(0, Q-1)
    sdiamond = random.randint(0, Q-1) 

    mu2    = random.randint(0, B2*B2) 
    muplus = random.randint(0, Q-1)
    mutimes = random.randint(0, Q-1)
    mudiamond = 0

    # ρ_i
    rho2    = random.randint(0, Q-1)
    rhoplus = random.randint(0, Q-1)
    rhotimes = random.randint(0, Q-1)
    rhodiamond = random.randint(0, Q-1)

    c2    = pedersen_commit_scalar(x2,    s2)
    cplus = pedersen_commit_scalar(xplus, splus)
    ctimes = pedersen_commit_scalar(xtimes, stimes)

    t2    = pedersen_commit_scalar(mu2,    rho2)
    tplus = pedersen_commit_scalar(muplus, rhoplus)
    ttimes = pedersen_commit_scalar(mutimes, rhotimes)
    tdiamond = pedersen_commit_scalar(mudiamond, rhodiamond)  # = h^{ρ◊}

    first_msg = {
        "mu_vec": mu_vec,
        "rho_vec": rho_vec,
        "T": T,

        "x2": x2,
        "xplus": xplus,
        "xtimes": xtimes,

        "s2": s2,
        "splus": splus,
        "stimes": stimes,
        "sdiamond": sdiamond,

        "mu2": mu2,
        "muplus": muplus,
        "mutimes": mutimes,
        "mudiamond": mudiamond,

        "rho2": rho2,
        "rhoplus": rhoplus,
        "rhotimes": rhotimes,
        "rhodiamond": rhodiamond,

        "c2": c2,
        "cplus": cplus,
        "ctimes": ctimes,

        "t2": t2,
        "tplus": tplus,
        "ttimes": ttimes,
        "tdiamond": tdiamond,
    }
    return first_msg

first_msg_l2 = prover_first_message_l2(x_l2, s_l2, B2)
print("First message keys:", list(first_msg_l2.keys()))


First message keys: ['mu_vec', 'rho_vec', 'T', 'x2', 'xplus', 'xtimes', 's2', 'splus', 'stimes', 'sdiamond', 'mu2', 'muplus', 'mutimes', 'mudiamond', 'rho2', 'rhoplus', 'rhotimes', 'rhodiamond', 'c2', 'cplus', 'ctimes', 't2', 'tplus', 'ttimes', 'tdiamond']


In [7]:
def verifier_challenge(max_c: int = 2**16) -> int:
    return random.randint(0, max_c)

c_l2 = verifier_challenge()
print("challenge c (l2) =", c_l2)


challenge c (l2) = 6915


In [8]:
def mod_q(x: int) -> int:
    return x % Q

def prover_response_l2(x, s, first_msg, c: int, B2: int):
    ell = len(x)

    mu_vec  = first_msg["mu_vec"]
    rho_vec = first_msg["rho_vec"]

    Rx = [c * x[i] + mu_vec[i] for i in range(ell)]
    Rs = [mod_q(c * s[i] + rho_vec[i]) for i in range(ell)]

    x2, xplus, xtimes = first_msg["x2"], first_msg["xplus"], first_msg["xtimes"]
    s2, splus, stimes = first_msg["s2"], first_msg["splus"], first_msg["stimes"]

    mu2, muplus, mutimes = first_msg["mu2"], first_msg["muplus"], first_msg["mutimes"]
    rho2, rhoplus, rhotimes = first_msg["rho2"], first_msg["rhoplus"], first_msg["rhotimes"]

    rhodiamond = first_msg["rhodiamond"]

    sdiamond = mod_q(- (c*c * s2 + c * splus + stimes))

    Rx2    = c * x2    + mu2
    Rxplus = c * xplus + muplus
    Rxtimes = c * xtimes + mutimes

    Rs2    = mod_q(c * s2    + rho2)
    Rsplus = mod_q(c * splus + rhoplus)
    Rstimes = mod_q(c * stimes + rhotimes)
    Rsdiamond = mod_q(c * sdiamond + rhodiamond)

    response = {
        "Rx": Rx,
        "Rs": Rs,

        "Rx2": Rx2,
        "Rxplus": Rxplus,
        "Rxtimes": Rxtimes,

        "Rs2": Rs2,
        "Rsplus": Rsplus,
        "Rstimes": Rstimes,
        "Rsdiamond": Rsdiamond,

        "sdiamond": sdiamond,
    }
    return response

resp_l2 = prover_response_l2(x_l2, s_l2, first_msg_l2, c_l2, B2)
print("Rx (first 4) =", resp_l2["Rx"])
print("Rs (first 4) =", resp_l2["Rs"])


Rx (first 4) = [155133913309133413001043325616459522325, 71314059558598922801074247446343151980, 103984834396116057106803486779806838787, 122821026825349065710274357827548206335]
Rs (first 4) = [141453058255176199825037434808596616705, 100762991469367518892974136174139794212, 1810975855477192919597105445368791323, 71026387204740106759937308620562131717]


In [9]:
def group_inv(a: int) -> int:
    return pow(a, P-2, P)

def verifier_check_l2(C, B2: int, first_msg, response, c: int) -> bool:
    ell = len(C)

    T = first_msg["T"]
    c2, cplus, ctimes = first_msg["c2"], first_msg["cplus"], first_msg["ctimes"]
    t2, tplus, ttimes, tdiamond = first_msg["t2"], first_msg["tplus"], first_msg["ttimes"], first_msg["tdiamond"]

    Rx = response["Rx"]
    Rs = response["Rs"]

    Rx2, Rxplus, Rxtimes = response["Rx2"], response["Rxplus"], response["Rxtimes"]
    Rs2, Rsplus, Rstimes, Rsdiamond = response["Rs2"], response["Rsplus"], response["Rstimes"], response["Rsdiamond"]

    inner_Rx = sum(v*v for v in Rx)
    num = pow(G, inner_Rx, P)
    den = pow(c2,    c*c, P)
    den = (den * pow(cplus, c, P)) % P
    den = (den * ctimes) % P
    cdiamond = (num * group_inv(den)) % P

    def check_scalar(Rx_i, Rs_i, t_i, c_i, name):
        left = (pow(G, Rx_i, P) * pow(H, Rs_i, P)) % P
        right = (t_i * pow(c_i, c, P)) % P
        if left != right:
            print(f"[l2 fail] scalar eq for {name}")
            return False
        return True

    if not check_scalar(Rx2,    Rs2,    t2,    c2,    "2"):
        return False
    if not check_scalar(Rxplus, Rsplus, tplus, cplus, "+"):
        return False
    if not check_scalar(Rxtimes,Rstimes,ttimes,ctimes,"×"):
        return False

    for j in range(ell):
        left  = (pow(G, Rx[j], P) * pow(H, Rs[j], P)) % P
        right = (T[j] * pow(C[j], c, P)) % P
        if left != right:
            print(f"[l2 fail] vector eq at coord {j}")
            return False

    left  = pow(H, Rsdiamond, P)
    right = (tdiamond * pow(cdiamond, c, P)) % P
    if left != right:
        print("[l2 fail] diamond eq")
        return False

    if Rx2 > (c + 1) * (B2 ** 2):
        print("[l2 fail] bound Rx2 >", (c+1) * (B2**2))
        return False

    return True

accepted_l2 = verifier_check_l2(C_l2, B2, first_msg_l2, resp_l2, c_l2)
print("Verifier accepts (l2)?", accepted_l2)


Verifier accepts (l2)? True
